In [ ]:
# 스트림릿용 CO2 배출량 시각화 코드 (더욱 예쁘고 인터랙티브한 버전)
streamlit_co2_code = '''
# CO2 배출량 데이터 시각화 (Streamlit) - 환경 친화적 대시보드
st.header('🌱 차량 CO2 배출량 환경 분석 대시보드')

# 환경 테마 CSS 적용
st.markdown("""
<style>
.eco-metric {
    background: linear-gradient(90deg, #2E8B57, #32CD32);
    padding: 1rem;
    border-radius: 10px;
    color: white;
    text-align: center;
    margin: 0.5rem 0;
}
.eco-title {
    color: #2E8B57;
    font-weight: bold;
}
</style>
""", unsafe_allow_html=True)

# 사이드바 필터링
st.sidebar.subheader('🎛️ 환경 분석 필터')
st.sidebar.markdown("친환경 차량 분석을 위한 필터를 설정하세요")

# 제조사 선택
selected_makes = st.sidebar.multiselect(
    '🏭 제조사 선택',
    co2_data['Make'].unique(),
    default=co2_data['Make'].value_counts().head(10).index.tolist()
)

# 차량 클래스 선택
selected_classes = st.sidebar.multiselect(
    '🚗 차량 클래스 선택',
    co2_data['Vehicle Class'].unique(),
    default=co2_data['Vehicle Class'].unique()
)

# CO2 배출량 범위 선택
co2_range = st.sidebar.slider(
    '🌍 CO2 배출량 범위 (g/km)',
    int(co2_data['CO2 Emissions(g/km)'].min()),
    int(co2_data['CO2 Emissions(g/km)'].max()),
    (int(co2_data['CO2 Emissions(g/km)'].min()), 
     int(co2_data['CO2 Emissions(g/km)'].max()))
)

# 엔진 크기 범위
engine_range = st.sidebar.slider(
    '🔧 엔진 크기 범위 (L)',
    float(co2_data['Engine Size(L)'].min()),
    float(co2_data['Engine Size(L)'].max()),
    (float(co2_data['Engine Size(L)'].min()), 
     float(co2_data['Engine Size(L)'].max())),
    step=0.1
)

# 데이터 필터링
filtered_co2 = co2_data[
    (co2_data['Make'].isin(selected_makes)) &
    (co2_data['Vehicle Class'].isin(selected_classes)) &
    (co2_data['CO2 Emissions(g/km)'].between(co2_range[0], co2_range[1])) &
    (co2_data['Engine Size(L)'].between(engine_range[0], engine_range[1]))
]

# 환경 지표 메트릭 카드
st.markdown('<div class="eco-title"><h3>🌍 환경 영향 지표</h3></div>', unsafe_allow_html=True)

col1, col2, col3, col4 = st.columns(4)

with col1:
    avg_co2 = filtered_co2['CO2 Emissions(g/km)'].mean()
    st.metric(
        "🌱 평균 CO2 배출량",
        f"{avg_co2:.1f} g/km",
        delta=f"{avg_co2 - co2_data['CO2 Emissions(g/km)'].mean():.1f} (vs 전체)"
    )

with col2:
    eco_friendly = len(filtered_co2[filtered_co2['CO2 Emissions(g/km)'] < 200])
    eco_ratio = (eco_friendly / len(filtered_co2) * 100) if len(filtered_co2) > 0 else 0
    st.metric(
        "♻️ 친환경 차량 비율",
        f"{eco_ratio:.1f}%",
        delta=f"{eco_friendly}대 (< 200g/km)"
    )

with col3:
    avg_mpg = filtered_co2['Fuel Consumption Comb (mpg)'].mean()
    st.metric(
        "⛽ 평균 연비",
        f"{avg_mpg:.1f} mpg",
        delta=f"{avg_mpg - co2_data['Fuel Consumption Comb (mpg)'].mean():.1f} (vs 전체)"
    )

with col4:
    total_vehicles = len(filtered_co2)
    st.metric(
        "🚗 분석 대상 차량",
        f"{total_vehicles:,}대",
        delta=f"{(total_vehicles/len(co2_data)*100):.1f}% of total"
    )

st.divider()

# 탭으로 구분된 상세 분석
tab1, tab2, tab3, tab4 = st.tabs([
    "📊 기본 분석", 
    "🔍 상관관계 분석", 
    "🏭 제조사 비교", 
    "🎯 친환경 차량 찾기"
])

with tab1:
    col1, col2 = st.columns(2)
    
    with col1:
        # CO2 배출량 분포
        fig1 = px.histogram(
            filtered_co2,
            x='CO2 Emissions(g/km)',
            nbins=30,
            title='🌍 CO2 배출량 분포',
            color_discrete_sequence=['#2E8B57']
        )
        fig1.add_vline(x=200, line_dash="dash", line_color="red", 
                      annotation_text="친환경 기준 (200g/km)")
        fig1.update_layout(
            xaxis_title='CO2 배출량 (g/km)',
            yaxis_title='차량 수',
            plot_bgcolor='rgba(0,0,0,0)'
        )
        st.plotly_chart(fig1, use_container_width=True)
    
    with col2:
        # 차량 클래스별 CO2 배출량
        fig2 = px.box(
            filtered_co2,
            x='Vehicle Class',
            y='CO2 Emissions(g/km)',
            title='🚗 차량 클래스별 CO2 배출량',
            color='Vehicle Class',
            color_discrete_sequence=px.colors.qualitative.Set3
        )
        fig2.update_layout(
            xaxis_title='차량 클래스',
            yaxis_title='CO2 배출량 (g/km)',
            plot_bgcolor='rgba(0,0,0,0)',
            showlegend=False
        )
        fig2.update_xaxes(tickangle=45)
        st.plotly_chart(fig2, use_container_width=True)

with tab2:
    # 엔진 크기 vs CO2 배출량 산점도
    fig3 = px.scatter(
        filtered_co2,
        x='Engine Size(L)',
        y='CO2 Emissions(g/km)',
        color='Cylinders',
        size='Fuel Consumption Comb (mpg)',
        hover_data=['Make', 'Model', 'Vehicle Class'],
        title='🔧 엔진 크기 vs CO2 배출량 (실린더 수별)',
        color_continuous_scale='RdYlGn_r'
    )
    fig3.update_layout(
        xaxis_title='엔진 크기 (L)',
        yaxis_title='CO2 배출량 (g/km)',
        plot_bgcolor='rgba(0,0,0,0)'
    )
    st.plotly_chart(fig3, use_container_width=True)
    
    # 연비 vs CO2 배출량 관계
    fig4 = px.scatter(
        filtered_co2,
        x='Fuel Consumption Comb (mpg)',
        y='CO2 Emissions(g/km)',
        color='Vehicle Class',
        title='⛽ 연비 vs CO2 배출량 관계',
        trendline='ols',
        hover_data=['Make', 'Model']
    )
    fig4.update_layout(
        xaxis_title='연비 (mpg)',
        yaxis_title='CO2 배출량 (g/km)',
        plot_bgcolor='rgba(0,0,0,0)'
    )
    st.plotly_chart(fig4, use_container_width=True)

with tab3:
    # 제조사별 환경 성능 비교
    if len(filtered_co2) > 0:
        make_stats = filtered_co2.groupby('Make').agg({
            'CO2 Emissions(g/km)': ['mean', 'count'],
            'Fuel Consumption Comb (mpg)': 'mean'
        }).round(1)
        
        make_stats.columns = ['평균_CO2배출량', '차량수', '평균_연비']
        make_stats = make_stats[make_stats['차량수'] >= 3].sort_values('평균_CO2배출량')
        
        # 환경 친화적 제조사 순위
        fig5 = px.bar(
            x=make_stats.index[:15],
            y=make_stats['평균_CO2배출량'][:15],
            title='🏭 제조사별 평균 CO2 배출량 (환경 친화적 순)',
            color=make_stats['평균_CO2배출량'][:15],
            color_continuous_scale='RdYlGn_r'
        )
        fig5.update_layout(
            xaxis_title='제조사',
            yaxis_title='평균 CO2 배출량 (g/km)',
            plot_bgcolor='rgba(0,0,0,0)',
            showlegend=False
        )
        fig5.update_xaxes(tickangle=45)
        st.plotly_chart(fig5, use_container_width=True)
        
        # 제조사별 상세 통계
        st.subheader("📈 제조사별 환경 성능 상세")
        st.dataframe(
            make_stats.head(15).style.background_gradient(
                subset=['평균_CO2배출량'], cmap='RdYlGn_r'
            ).format({
                '평균_CO2배출량': '{:.1f} g/km',
                '평균_연비': '{:.1f} mpg',
                '차량수': '{:.0f}대'
            }),
            use_container_width=True
        )

with tab4:
    st.subheader("🎯 친환경 차량 추천 시스템")
    
    # 친환경 기준 설정
    col1, col2 = st.columns(2)
    
    with col1:
        max_co2 = st.number_input(
            "최대 CO2 배출량 기준 (g/km)",
            min_value=100,
            max_value=300,
            value=200,
            step=10
        )
    
    with col2:
        min_mpg = st.number_input(
            "최소 연비 기준 (mpg)",
            min_value=15,
            max_value=50,
            value=30,
            step=1
        )
    
    # 친환경 차량 필터링
    eco_vehicles = filtered_co2[
        (filtered_co2['CO2 Emissions(g/km)'] <= max_co2) &
        (filtered_co2['Fuel Consumption Comb (mpg)'] >= min_mpg)
    ].sort_values('CO2 Emissions(g/km)')
    
    if len(eco_vehicles) > 0:
        st.success(f"🌟 기준에 맞는 친환경 차량 {len(eco_vehicles)}대 발견!")
        
        # 추천 차량 시각화
        fig6 = px.scatter(
            eco_vehicles.head(20),
            x='Fuel Consumption Comb (mpg)',
            y='CO2 Emissions(g/km)',
            size='Engine Size(L)',
            color='Make',
            hover_data=['Model', 'Vehicle Class'],
            title=f'🌱 추천 친환경 차량 TOP 20',
            size_max=15
        )
        fig6.update_layout(
            xaxis_title='연비 (mpg)',
            yaxis_title='CO2 배출량 (g/km)',
            plot_bgcolor='rgba(0,0,0,0)'
        )
        st.plotly_chart(fig6, use_container_width=True)
        
        # 추천 차량 리스트
        st.subheader("🏆 추천 친환경 차량 TOP 10")
        recommendation_df = eco_vehicles.head(10)[
            ['Make', 'Model', 'Vehicle Class', 'Engine Size(L)', 
             'CO2 Emissions(g/km)', 'Fuel Consumption Comb (mpg)']
        ].copy()
        
        # 환경 점수 계산 (CO2는 낮을수록, 연비는 높을수록 좋음)
        recommendation_df['환경점수'] = (
            (50 - recommendation_df['CO2 Emissions(g/km)'] / 10) +
            (recommendation_df['Fuel Consumption Comb (mpg)'] * 2)
        ).round(1)
        
        st.dataframe(
            recommendation_df.style.background_gradient(
                subset=['환경점수'], cmap='Greens'
            ).format({
                'Engine Size(L)': '{:.1f}L',
                'CO2 Emissions(g/km)': '{:.0f} g/km',
                'Fuel Consumption Comb (mpg)': '{:.0f} mpg',
                '환경점수': '{:.1f}점'
            }),
            use_container_width=True
        )
    else:
        st.warning("⚠️ 설정한 기준에 맞는 차량이 없습니다. 기준을 조정해주세요.")

# 환경 팁
st.markdown("""
---
### 🌱 친환경 운전 팁
- **200g/km 이하**: 우수한 환경 성능
- **하이브리드 차량**: 'HYBRID' 모델명 포함 차량 우선 고려
- **작은 엔진**: 2.0L 이하 엔진이 일반적으로 친환경적
- **연비 중심**: 30mpg 이상 차량 추천
""")
'''

print("\\n스트림릿용 CO2 배출량 시각화 코드:")
print("환경 친화적 테마와 인터랙티브 분석이 포함된 고급 대시보드입니다!")

In [12]:
# CO2 배출량 데이터의 예쁜 시각화 분석
print("=== CO2 배출량 데이터 상세 시각화 ===")

# 1. 전체적인 시각화 레이아웃 설정
fig, axes = plt.subplots(3, 2, figsize=(18, 15))
fig.suptitle('🌱 Vehicle CO2 Emissions Analysis Dashboard', fontsize=20, fontweight='bold', y=0.98)

# 색상 팔레트 설정 (환경 친화적인 색상)
colors_env = ['#2E8B57', '#32CD32', '#228B22', '#006400', '#8FBC8F', '#90EE90']
colors_emission = ['#FF6B6B', '#4ECDC4', '#45B7D1', '#96CEB4', '#FFEAA7', '#DDA0DD']

# 1. CO2 배출량 분포 (히스토그램 + KDE)
sns.histplot(co2_data['CO2 Emissions(g/km)'], bins=50, kde=True, 
             color='#2E8B57', alpha=0.7, ax=axes[0,0])
axes[0,0].set_title('📊 CO2 Emissions Distribution', fontsize=14, fontweight='bold')
axes[0,0].set_xlabel('CO2 Emissions (g/km)')
axes[0,0].set_ylabel('Frequency')
axes[0,0].grid(True, alpha=0.3)

# 평균선 추가
mean_co2 = co2_data['CO2 Emissions(g/km)'].mean()
axes[0,0].axvline(mean_co2, color='red', linestyle='--', linewidth=2, 
                  label=f'평균: {mean_co2:.1f} g/km')
axes[0,0].legend()

# 2. 차량 클래스별 CO2 배출량 박스플롯
sns.boxplot(data=co2_data, y='Vehicle Class', x='CO2 Emissions(g/km)', 
            palette='viridis', ax=axes[0,1])
axes[0,1].set_title('🚗 CO2 Emissions by Vehicle Class', fontsize=14, fontweight='bold')
axes[0,1].set_xlabel('CO2 Emissions (g/km)')
axes[0,1].grid(True, alpha=0.3)

# 3. 엔진 크기 vs CO2 배출량 산점도
scatter = axes[1,0].scatter(co2_data['Engine Size(L)'], co2_data['CO2 Emissions(g/km)'], 
                           c=co2_data['Cylinders'], cmap='RdYlGn_r', alpha=0.6, s=30)
axes[1,0].set_title('🔧 Engine Size vs CO2 Emissions', fontsize=14, fontweight='bold')
axes[1,0].set_xlabel('Engine Size (L)')
axes[1,0].set_ylabel('CO2 Emissions (g/km)')
axes[1,0].grid(True, alpha=0.3)

# 컬러바 추가
cbar = plt.colorbar(scatter, ax=axes[1,0])
cbar.set_label('Number of Cylinders', fontsize=10)

# 4. 연비 vs CO2 배출량 관계
axes[1,1].scatter(co2_data['Fuel Consumption Comb (mpg)'], co2_data['CO2 Emissions(g/km)'], 
                  alpha=0.6, color='#FF6B6B', s=30)
axes[1,1].set_title('⛽ Fuel Efficiency vs CO2 Emissions', fontsize=14, fontweight='bold')
axes[1,1].set_xlabel('Fuel Consumption Combined (mpg)')
axes[1,1].set_ylabel('CO2 Emissions (g/km)')
axes[1,1].grid(True, alpha=0.3)

# 추세선 추가
z = np.polyfit(co2_data['Fuel Consumption Comb (mpg)'], co2_data['CO2 Emissions(g/km)'], 1)
p = np.poly1d(z)
axes[1,1].plot(co2_data['Fuel Consumption Comb (mpg)'], p(co2_data['Fuel Consumption Comb (mpg)']), 
               "r--", alpha=0.8, linewidth=2)

# 5. 제조사별 평균 CO2 배출량 (상위 15개)
top_makes = co2_data['Make'].value_counts().head(15).index
make_co2_avg = co2_data[co2_data['Make'].isin(top_makes)].groupby('Make')['CO2 Emissions(g/km)'].mean().sort_values(ascending=True)

bars = axes[2,0].barh(range(len(make_co2_avg)), make_co2_avg.values, 
                      color=plt.cm.RdYlGn_r(make_co2_avg.values / make_co2_avg.max()))
axes[2,0].set_yticks(range(len(make_co2_avg)))
axes[2,0].set_yticklabels(make_co2_avg.index, fontsize=10)
axes[2,0].set_title('🏭 Average CO2 Emissions by Manufacturer', fontsize=14, fontweight='bold')
axes[2,0].set_xlabel('Average CO2 Emissions (g/km)')
axes[2,0].grid(True, alpha=0.3, axis='x')

# 값 표시
for i, bar in enumerate(bars):
    width = bar.get_width()
    axes[2,0].text(width + 2, bar.get_y() + bar.get_height()/2, 
                   f'{width:.0f}', ha='left', va='center', fontsize=9)

# 6. 실린더 수별 CO2 배출량 분포
cylinder_co2 = co2_data.groupby('Cylinders')['CO2 Emissions(g/km)'].mean().sort_index()
bars2 = axes[2,1].bar(cylinder_co2.index, cylinder_co2.values, 
                      color=plt.cm.plasma(np.linspace(0, 1, len(cylinder_co2))))
axes[2,1].set_title('🔩 CO2 Emissions by Number of Cylinders', fontsize=14, fontweight='bold')
axes[2,1].set_xlabel('Number of Cylinders')
axes[2,1].set_ylabel('Average CO2 Emissions (g/km)')
axes[2,1].grid(True, alpha=0.3)

# 값 표시
for bar in bars2:
    height = bar.get_height()
    axes[2,1].text(bar.get_x() + bar.get_width()/2., height + 2,
                   f'{height:.0f}', ha='center', va='bottom', fontsize=10)

plt.tight_layout()
plt.show()

print("\\n🌟 인사이트:")
print(f"• 가장 친환경적인 제조사: {make_co2_avg.index[0]} (평균 {make_co2_avg.iloc[0]:.1f} g/km)")
print(f"• 연비와 CO2 배출량은 강한 음의 상관관계를 보입니다")
print(f"• 엔진 크기가 클수록 CO2 배출량이 증가하는 경향")
print(f"• 실린더 수가 많을수록 CO2 배출량 증가")

=== CO2 배출량 데이터 상세 시각화 ===


NameError: name 'co2_data' is not defined

In [13]:
# CO2 배출량 데이터 로드 및 분석
co2_data = pd.read_csv(data_path + "CO2_Emissions.csv")

print("=== CO2 배출량 데이터 기본 분석 ===")
print(f"총 차량 수: {len(co2_data):,}대")
print(f"제조사 수: {co2_data['Make'].nunique()}개")
print(f"차량 클래스 수: {co2_data['Vehicle Class'].nunique()}개")
print(f"평균 CO2 배출량: {co2_data['CO2 Emissions(g/km)'].mean():.1f} g/km")
print(f"최대 CO2 배출량: {co2_data['CO2 Emissions(g/km)'].max()} g/km")
print(f"최소 CO2 배출량: {co2_data['CO2 Emissions(g/km)'].min()} g/km")

print(f"\n제조사별 차량 수:")
print(co2_data['Make'].value_counts().head(10))

print(f"\n차량 클래스별 분포:")
print(co2_data['Vehicle Class'].value_counts())

# 기본 데이터 정보
co2_data.head()

=== CO2 배출량 데이터 기본 분석 ===
총 차량 수: 7,385대
제조사 수: 42개
차량 클래스 수: 16개
평균 CO2 배출량: 250.6 g/km
최대 CO2 배출량: 522 g/km
최소 CO2 배출량: 96 g/km

제조사별 차량 수:
Make
FORD             628
CHEVROLET        588
BMW              527
MERCEDES-BENZ    419
PORSCHE          376
TOYOTA           330
GMC              328
AUDI             286
NISSAN           259
JEEP             251
Name: count, dtype: int64

차량 클래스별 분포:
Vehicle Class
SUV - SMALL                 1217
MID-SIZE                    1133
COMPACT                     1022
SUV - STANDARD               735
FULL-SIZE                    639
SUBCOMPACT                   606
PICKUP TRUCK - STANDARD      538
TWO-SEATER                   460
MINICOMPACT                  326
STATION WAGON - SMALL        252
PICKUP TRUCK - SMALL         159
MINIVAN                       80
SPECIAL PURPOSE VEHICLE       77
VAN - PASSENGER               66
STATION WAGON - MID-SIZE      53
VAN - CARGO                   22
Name: count, dtype: int64


,Make,Model,Vehicle Class,Engine Size(L),Cylinders,Transmission,Fuel Type,Fuel Consumption City (L/100 km),Fuel Consumption Hwy (L/100 km),Fuel Consumption Comb (L/100 km),Fuel Consumption Comb (mpg),CO2 Emissions(g/km)
0,ACURA,ILX,COMPACT,2.0,4,AS5,Z,9.9,6.7,8.5,33,196
1,ACURA,ILX,COMPACT,2.4,4,M6,Z,11.2,7.7,9.6,29,221
2,ACURA,ILX HYBRID,COMPACT,1.5,4,AV7,Z,6.0,5.8,5.9,48,136
3,ACURA,MDX 4WD,SUV - SMALL,3.5,6,AS6,Z,12.7,9.1,11.1,25,255
4,ACURA,RDX AWD,SUV - SMALL,3.5,6,AS6,Z,12.1,8.7,10.6,27,244


### 5.4 CO2 배출량 데이터 시각화 (새로 추가!)

# 15주차 클라우드 기반 데이터 시각화 - 스트림릿 구성 요소 학습 및 실습
- 작성일: 2025년 8월 19일
- 스트림릿 데이터 시각화 실습 노트북


## 1. 환경 설정 및 라이브러리 임포트

스트림릿 애플리케이션 개발에 필요한 라이브러리들을 임포트합니다.

In [1]:
# 필수 라이브러리 임포트
import streamlit as st
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
import plotly.graph_objects as go
from datetime import datetime, time
import warnings
warnings.filterwarnings('ignore')

# 한글 폰트 설정 (matplotlib용)
plt.rcParams['font.family'] = 'DejaVu Sans'
plt.rcParams['axes.unicode_minus'] = False

print("라이브러리 임포트 완료!")

라이브러리 임포트 완료!


## 2. 데이터셋 로드 및 탐색

제공된 3개의 데이터셋을 로드하고 기본 구조를 파악합니다.

In [2]:
# 데이터셋 경로 설정
data_path = "C:/githome/15week_cloud-data-viz/dataset/"

# 1. Airbnb 주식 데이터 로드
abnb_stock = pd.read_csv(data_path + "ABNB_stock.csv")
abnb_stock['Date'] = pd.to_datetime(abnb_stock['Date'])

# 2. 전기차 충전 데이터 로드
ev_charge = pd.read_csv(data_path + "EV_charge.csv")

# 3. 의료비 데이터 로드

medical_cost = pd.read_csv(data_path + "medical_cost.csv")

print("=== 데이터셋 기본 정보 ===")
print(f"\n1. ABNB Stock 데이터: {abnb_stock.shape}")
print(abnb_stock.head())
print(f"\n2. EV Charge 데이터: {ev_charge.shape}")
print(ev_charge.head())
print(f"\n3. Medical Cost 데이터: {medical_cost.shape}")
print(medical_cost.head())

=== 데이터셋 기본 정보 ===

1. ABNB Stock 데이터: (454, 7)
        Date        Open        High         Low       Close   Adj Close  \
0 2020-12-11  146.550003  151.500000  135.100006  139.250000  139.250000   
1 2020-12-14  135.000000  135.300003  125.160004  130.000000  130.000000   
2 2020-12-15  126.690002  127.599998  121.500000  124.800003  124.800003   
3 2020-12-16  125.830002  142.000000  124.910004  137.990005  137.990005   
4 2020-12-17  143.000000  152.449997  142.669998  147.050003  147.050003   

     Volume  
0  26980800  
1  16966100  
2  10914400  
3  20409600  
4  15054700  

2. EV Charge 데이터: (3395, 24)
   sessionId  kwhTotal  dollars              created                ended  \
0    1366563      7.78     0.00  0014-11-18 15:40:26  0014-11-18 17:11:04   
1    3075723      9.74     0.00  0014-11-19 17:40:26  0014-11-19 19:51:04   
2    4228788      6.76     0.58  0014-11-21 12:05:46  0014-11-21 16:46:04   
3    3173284      6.17     0.00  0014-12-03 19:16:12  0014-12-03 21:02:18

## 3. 스트림릿 기본 구성 요소 실습

### 3.1 텍스트 출력 요소 (Text Elements)

In [4]:
# 스트림릿 텍스트 출력 예제 코드 (실제 실행은 streamlit run으로)
streamlit_text_code = '''
import streamlit as st

# 제목 및 헤더 설정
st.title('📊 스트림릿 데이터 시각화 대시보드')
st.header('🎯 오늘의 학습 목표')
st.subheader('스트림릿 구성 요소 마스터하기')

# 마크다운 텍스트
st.markdown(
    """
    ### 스트림릿의 주요 특징:
    - :red[빠른 프로토타이핑] 가능
    - :blue[Python 코드만으로] 웹앱 구축
    - :green[실시간 업데이트] 지원
    
    **굵은 글씨**와 *기울임 글씨*도 지원합니다.
    """
)

# 일반 텍스트
st.text('이것은 일반 텍스트입니다. 마크다운 문법이 적용되지 않습니다.')

# 구분선
st.divider()
'''

print("스트림릿 텍스트 출력 예제 코드:")
print(streamlit_text_code)

스트림릿 텍스트 출력 예제 코드:

import streamlit as st

# 제목 및 헤더 설정
st.title('📊 스트림릿 데이터 시각화 대시보드')
st.header('🎯 오늘의 학습 목표')
st.subheader('스트림릿 구성 요소 마스터하기')

# 마크다운 텍스트
st.markdown(
    """
    ### 스트림릿의 주요 특징:
    - :red[빠른 프로토타이핑] 가능
    - :blue[Python 코드만으로] 웹앱 구축
    - :green[실시간 업데이트] 지원

    **굵은 글씨**와 *기울임 글씨*도 지원합니다.
    """
)

# 일반 텍스트
st.text('이것은 일반 텍스트입니다. 마크다운 문법이 적용되지 않습니다.')

# 구분선
st.divider()



### 3.2 데이터 표시 요소 (Data Display Elements)

In [5]:
# 스트림릿 데이터 표시 예제 코드
streamlit_data_code = '''
import streamlit as st
import pandas as pd

# 데이터프레임 표시
st.header('📋 데이터프레임 표시')

# 정적 테이블 (st.table)
st.subheader('정적 테이블 (st.table)')
sample_data = {
    '이름': ['김철수', '이영희', '박민수'],
    '나이': [25, 30, 35],
    '직업': ['개발자', '디자이너', '기획자']
}
df_sample = pd.DataFrame(sample_data)
st.table(df_sample)

# 인터랙티브 데이터프레임 (st.dataframe)
st.subheader('인터랙티브 데이터프레임 (st.dataframe)')
st.dataframe(medical_cost.head(10), use_container_width=True)

# 코드 블록 표시
st.subheader('코드 블록 (st.code)')
st.code(
    """
    import pandas as pd
    import streamlit as st
    
    # 데이터 로드
    df = pd.read_csv('data.csv')
    st.dataframe(df)
    """, 
    language='python'
)

# JSON 데이터 표시
st.subheader('JSON 데이터 (st.json)')
json_data = {
    'project': 'Streamlit Dashboard',
    'version': '1.0',
    'features': ['data visualization', 'interactive widgets', 'real-time updates']
}
st.json(json_data)
'''

print("스트림릿 데이터 표시 예제 코드:")
print(streamlit_data_code)

스트림릿 데이터 표시 예제 코드:

import streamlit as st
import pandas as pd

# 데이터프레임 표시
st.header('📋 데이터프레임 표시')

# 정적 테이블 (st.table)
st.subheader('정적 테이블 (st.table)')
sample_data = {
    '이름': ['김철수', '이영희', '박민수'],
    '나이': [25, 30, 35],
    '직업': ['개발자', '디자이너', '기획자']
}
df_sample = pd.DataFrame(sample_data)
st.table(df_sample)

# 인터랙티브 데이터프레임 (st.dataframe)
st.subheader('인터랙티브 데이터프레임 (st.dataframe)')
st.dataframe(medical_cost.head(10), use_container_width=True)

# 코드 블록 표시
st.subheader('코드 블록 (st.code)')
st.code(
    """
    import pandas as pd
    import streamlit as st

    # 데이터 로드
    df = pd.read_csv('data.csv')
    st.dataframe(df)
    """, 
    language='python'
)

# JSON 데이터 표시
st.subheader('JSON 데이터 (st.json)')
json_data = {
    'project': 'Streamlit Dashboard',
    'version': '1.0',
    'features': ['data visualization', 'interactive widgets', 'real-time updates']
}
st.json(json_data)



### 3.3 메트릭 및 KPI 표시 (st.metric)

In [6]:
# 메트릭 표시 예제 코드
streamlit_metric_code = '''
import streamlit as st
import pandas as pd

st.header('📊 핵심 지표 (KPI) 대시보드')

# 3개 컬럼으로 메트릭 배치
col1, col2, col3 = st.columns(3)

with col1:
    # ABNB 주식 현재가 및 변동
    current_price = abnb_stock['Close'].iloc[-1]
    prev_price = abnb_stock['Close'].iloc[-2]
    price_change = current_price - prev_price
    
    st.metric(
        label="📈 ABNB 주가",
        value=f"${current_price:.2f}",
        delta=f"{price_change:.2f}"
    )

with col2:
    # 평균 의료비
    avg_charges = medical_cost['charges'].mean()
    median_charges = medical_cost['charges'].median()
    charges_diff = avg_charges - median_charges
    
    st.metric(
        label="🏥 평균 의료비",
        value=f"${avg_charges:,.0f}",
        delta=f"{charges_diff:,.0f} (vs 중앙값)"
    )

with col3:
    # 총 충전 에너지
    total_kwh = ev_charge['kwhTotal'].sum()
    avg_kwh = ev_charge['kwhTotal'].mean()
    
    st.metric(
        label="⚡ 총 충전량",
        value=f"{total_kwh:,.1f} kWh",
        delta=f"평균: {avg_kwh:.1f} kWh",
        delta_color="off"
    )
'''

print("스트림릿 메트릭 표시 예제 코드:")
print(streamlit_metric_code)

스트림릿 메트릭 표시 예제 코드:

import streamlit as st
import pandas as pd

st.header('📊 핵심 지표 (KPI) 대시보드')

# 3개 컬럼으로 메트릭 배치
col1, col2, col3 = st.columns(3)

with col1:
    # ABNB 주식 현재가 및 변동
    current_price = abnb_stock['Close'].iloc[-1]
    prev_price = abnb_stock['Close'].iloc[-2]
    price_change = current_price - prev_price

    st.metric(
        label="📈 ABNB 주가",
        value=f"${current_price:.2f}",
        delta=f"{price_change:.2f}"
    )

with col2:
    # 평균 의료비
    avg_charges = medical_cost['charges'].mean()
    median_charges = medical_cost['charges'].median()
    charges_diff = avg_charges - median_charges

    st.metric(
        label="🏥 평균 의료비",
        value=f"${avg_charges:,.0f}",
        delta=f"{charges_diff:,.0f} (vs 중앙값)"
    )

with col3:
    # 총 충전 에너지
    total_kwh = ev_charge['kwhTotal'].sum()
    avg_kwh = ev_charge['kwhTotal'].mean()

    st.metric(
        label="⚡ 총 충전량",
        value=f"{total_kwh:,.1f} kWh",
        delta=f"평균: {avg_kwh:.1f} kWh",
        del

## 4. 사용자 입력 위젯 (Input Widgets) 실습

In [7]:
# 사용자 입력 위젯 예제 코드
streamlit_input_code = '''
import streamlit as st
import pandas as pd

st.header('🎛️ 사용자 입력 위젯')

# 사이드바에 컨트롤 패널 생성
st.sidebar.header('📋 컨트롤 패널')

# 1. 버튼
st.subheader('1. 버튼 (Button)')
if st.button('데이터 새로고침', type='primary'):
    st.success('데이터가 새로고침되었습니다!')

# 2. 체크박스
st.subheader('2. 체크박스 (Checkbox)')
show_raw_data = st.checkbox('원본 데이터 표시')
if show_raw_data:
    st.dataframe(medical_cost.head())

# 3. 라디오 버튼
st.subheader('3. 라디오 버튼 (Radio)')
chart_type = st.radio(
    '차트 유형 선택:',
    ['히스토그램', '박스플롯', '산점도']
)
st.write(f'선택된 차트: {chart_type}')

# 4. 선택박스
st.subheader('4. 선택박스 (Selectbox)')
selected_column = st.selectbox(
    '분석할 컬럼 선택:',
    medical_cost.select_dtypes(include=['float64', 'int64']).columns
)

# 5. 다중 선택
st.subheader('5. 다중 선택 (Multiselect)')
selected_regions = st.multiselect(
    '지역 선택:',
    medical_cost['region'].unique(),
    default=medical_cost['region'].unique()[:2]
)

# 6. 슬라이더
st.subheader('6. 슬라이더 (Slider)')
age_range = st.slider(
    '나이 범위 선택:',
    min_value=int(medical_cost['age'].min()),
    max_value=int(medical_cost['age'].max()),
    value=(25, 50)
)

# 7. 텍스트 입력
st.subheader('7. 텍스트 입력 (Text Input)')
user_name = st.text_input('이름을 입력하세요:', placeholder='홍길동')
if user_name:
    st.write(f'안녕하세요, {user_name}님!')

# 8. 숫자 입력
st.subheader('8. 숫자 입력 (Number Input)')
threshold = st.number_input(
    '임계값 설정:',
    min_value=0.0,
    max_value=100.0,
    value=50.0,
    step=0.1
)
'''

print("스트림릿 입력 위젯 예제 코드:")
print(streamlit_input_code)

스트림릿 입력 위젯 예제 코드:

import streamlit as st
import pandas as pd

st.header('🎛️ 사용자 입력 위젯')

# 사이드바에 컨트롤 패널 생성
st.sidebar.header('📋 컨트롤 패널')

# 1. 버튼
st.subheader('1. 버튼 (Button)')
if st.button('데이터 새로고침', type='primary'):
    st.success('데이터가 새로고침되었습니다!')

# 2. 체크박스
st.subheader('2. 체크박스 (Checkbox)')
show_raw_data = st.checkbox('원본 데이터 표시')
if show_raw_data:
    st.dataframe(medical_cost.head())

# 3. 라디오 버튼
st.subheader('3. 라디오 버튼 (Radio)')
chart_type = st.radio(
    '차트 유형 선택:',
    ['히스토그램', '박스플롯', '산점도']
)
st.write(f'선택된 차트: {chart_type}')

# 4. 선택박스
st.subheader('4. 선택박스 (Selectbox)')
selected_column = st.selectbox(
    '분석할 컬럼 선택:',
    medical_cost.select_dtypes(include=['float64', 'int64']).columns
)

# 5. 다중 선택
st.subheader('5. 다중 선택 (Multiselect)')
selected_regions = st.multiselect(
    '지역 선택:',
    medical_cost['region'].unique(),
    default=medical_cost['region'].unique()[:2]
)

# 6. 슬라이더
st.subheader('6. 슬라이더 (Slider)')
age_range = st.slider(
    '나이 범위 선택:',
    min_valu

## 5. 데이터셋별 시각화 예제

### 5.1 ABNB 주식 데이터 시각화

In [8]:
# ABNB 주식 시각화 분석
print("=== ABNB 주식 데이터 기본 분석 ===")
print(f"기간: {abnb_stock['Date'].min()} ~ {abnb_stock['Date'].max()}")
print(f"최고가: ${abnb_stock['High'].max():.2f}")
print(f"최저가: ${abnb_stock['Low'].min():.2f}")
print(f"평균 거래량: {abnb_stock['Volume'].mean():,.0f}")

# Matplotlib으로 기본 차트 생성
fig, axes = plt.subplots(2, 2, figsize=(15, 10))
fig.suptitle('ABNB Stock Analysis', fontsize=16, fontweight='bold')

# 1. 종가 추이
axes[0,0].plot(abnb_stock['Date'], abnb_stock['Close'], color='blue', linewidth=2)
axes[0,0].set_title('Close Price Trend')
axes[0,0].set_ylabel('Price ($)')
axes[0,0].grid(True, alpha=0.3)

# 2. 거래량
axes[0,1].bar(abnb_stock['Date'], abnb_stock['Volume'], color='orange', alpha=0.7)
axes[0,1].set_title('Trading Volume')
axes[0,1].set_ylabel('Volume')

# 3. 고가-저가 범위
axes[1,0].fill_between(abnb_stock['Date'], abnb_stock['Low'], abnb_stock['High'], 
                       alpha=0.3, color='green', label='High-Low Range')
axes[1,0].plot(abnb_stock['Date'], abnb_stock['Close'], color='red', label='Close')
axes[1,0].set_title('Price Range (High-Low)')
axes[1,0].set_ylabel('Price ($)')
axes[1,0].legend()

# 4. 일일 변동률
daily_return = ((abnb_stock['Close'] - abnb_stock['Open']) / abnb_stock['Open'] * 100)
colors = ['red' if x < 0 else 'green' for x in daily_return]
axes[1,1].bar(abnb_stock['Date'], daily_return, color=colors, alpha=0.7)
axes[1,1].set_title('Daily Return (%)')
axes[1,1].set_ylabel('Return (%)')
axes[1,1].axhline(y=0, color='black', linestyle='-', linewidth=0.5)

plt.tight_layout()
plt.show()

# 스트림릿용 ABNB 시각화 코드
streamlit_abnb_code = '''
# ABNB 주식 데이터 시각화 (Streamlit)
st.header('📈 ABNB 주식 데이터 분석')

# 사이드바에서 날짜 범위 선택
st.sidebar.subheader('📅 기간 설정')
start_date = st.sidebar.date_input('시작일', abnb_stock['Date'].min())
end_date = st.sidebar.date_input('종료일', abnb_stock['Date'].max())

# 날짜 필터링
filtered_abnb = abnb_stock[
    (abnb_stock['Date'] >= pd.to_datetime(start_date)) & 
    (abnb_stock['Date'] <= pd.to_datetime(end_date))
]

# Plotly로 인터랙티브 차트 생성
fig = go.Figure()

# 캔들스틱 차트
fig.add_trace(go.Candlestick(
    x=filtered_abnb['Date'],
    open=filtered_abnb['Open'],
    high=filtered_abnb['High'],
    low=filtered_abnb['Low'],
    close=filtered_abnb['Close'],
    name='ABNB Stock'
))

fig.update_layout(
    title='ABNB Stock Candlestick Chart',
    xaxis_title='Date',
    yaxis_title='Price ($)',
    height=600
)

st.plotly_chart(fig, use_container_width=True)

# 거래량 차트
volume_fig = px.bar(
    filtered_abnb, 
    x='Date', 
    y='Volume',
    title='Trading Volume Over Time',
    color='Volume',
    color_continuous_scale='viridis'
)
st.plotly_chart(volume_fig, use_container_width=True)
'''

print("\n스트림릿용 ABNB 시각화 코드:")
print(streamlit_abnb_code)

=== ABNB 주식 데이터 기본 분석 ===
기간: 2020-12-11 00:00:00 ~ 2022-09-30 00:00:00
최고가: $219.94
최저가: $86.71
평균 거래량: 6,690,328

스트림릿용 ABNB 시각화 코드:

# ABNB 주식 데이터 시각화 (Streamlit)
st.header('📈 ABNB 주식 데이터 분석')

# 사이드바에서 날짜 범위 선택
st.sidebar.subheader('📅 기간 설정')
start_date = st.sidebar.date_input('시작일', abnb_stock['Date'].min())
end_date = st.sidebar.date_input('종료일', abnb_stock['Date'].max())

# 날짜 필터링
filtered_abnb = abnb_stock[
    (abnb_stock['Date'] >= pd.to_datetime(start_date)) & 
    (abnb_stock['Date'] <= pd.to_datetime(end_date))
]

# Plotly로 인터랙티브 차트 생성
fig = go.Figure()

# 캔들스틱 차트
fig.add_trace(go.Candlestick(
    x=filtered_abnb['Date'],
    open=filtered_abnb['Open'],
    high=filtered_abnb['High'],
    low=filtered_abnb['Low'],
    close=filtered_abnb['Close'],
    name='ABNB Stock'
))

fig.update_layout(
    title='ABNB Stock Candlestick Chart',
    xaxis_title='Date',
    yaxis_title='Price ($)',
    height=600
)

st.plotly_chart(fig, use_container_width=True)

# 거래량 차트
volume_fig = p

### 5.2 EV 충전 데이터 시각화

In [9]:
# EV 충전 데이터 분석
print("=== EV 충전 데이터 기본 분석 ===")
print(f"총 충전 세션: {len(ev_charge):,}건")
print(f"총 충전량: {ev_charge['kwhTotal'].sum():,.1f} kWh")
print(f"평균 충전량: {ev_charge['kwhTotal'].mean():.2f} kWh")
print(f"평균 충전 시간: {ev_charge['chargeTimeHrs'].mean():.2f} 시간")

# 요일별 분석을 위한 데이터 준비
weekday_cols = ['Mon', 'Tues', 'Wed', 'Thurs', 'Fri', 'Sat', 'Sun']
weekday_data = []
for col in weekday_cols:
    count = ev_charge[col].sum()
    avg_kwh = ev_charge[ev_charge[col] == 1]['kwhTotal'].mean() if count > 0 else 0
    weekday_data.append({'Day': col, 'Count': count, 'Avg_kWh': avg_kwh})

weekday_df = pd.DataFrame(weekday_data)

# Seaborn으로 시각화
fig, axes = plt.subplots(2, 2, figsize=(15, 10))
fig.suptitle('EV Charging Data Analysis', fontsize=16, fontweight='bold')

# 1. 충전량 분포
sns.histplot(ev_charge['kwhTotal'], bins=50, kde=True, ax=axes[0,0])
axes[0,0].set_title('Distribution of Charging Amount (kWh)')
axes[0,0].set_xlabel('kWh Total')

# 2. 충전 시간 vs 충전량
sns.scatterplot(data=ev_charge, x='chargeTimeHrs', y='kwhTotal', 
                alpha=0.6, ax=axes[0,1])
axes[0,1].set_title('Charging Time vs Amount')
axes[0,1].set_xlabel('Charge Time (Hours)')
axes[0,1].set_ylabel('kWh Total')

# 3. 요일별 충전 빈도
sns.barplot(data=weekday_df, x='Day', y='Count', ax=axes[1,0])
axes[1,0].set_title('Charging Frequency by Day of Week')
axes[1,0].set_ylabel('Number of Sessions')
axes[1,0].tick_params(axis='x', rotation=45)

# 4. 시간대별 충전 시작 분포
ev_charge['startTime'].value_counts().sort_index().plot(kind='bar', ax=axes[1,1])
axes[1,1].set_title('Charging Start Time Distribution')
axes[1,1].set_xlabel('Start Time (Hour)')
axes[1,1].set_ylabel('Frequency')
axes[1,1].tick_params(axis='x', rotation=45)

plt.tight_layout()
plt.show()

# 스트림릿용 EV 충전 시각화 코드
streamlit_ev_code = '''
# EV 충전 데이터 시각화 (Streamlit)
st.header('⚡ 전기차 충전 패턴 분석')

# 필터링 옵션
st.sidebar.subheader('🔧 필터 설정')
min_kwh = st.sidebar.slider('최소 충전량 (kWh)', 0.0, float(ev_charge['kwhTotal'].max()), 0.0)
selected_platform = st.sidebar.multiselect(
    '플랫폼 선택',
    ev_charge['platform'].unique(),
    default=ev_charge['platform'].unique()
)

# 데이터 필터링
filtered_ev = ev_charge[
    (ev_charge['kwhTotal'] >= min_kwh) &
    (ev_charge['platform'].isin(selected_platform))
]

# 탭으로 구분된 시각화
tab1, tab2, tab3 = st.tabs(["📊 기본 분석", "📈 패턴 분석", "🗺️ 위치 분석"])

with tab1:
    col1, col2 = st.columns(2)
    
    with col1:
        # 충전량 분포
        fig1 = px.histogram(
            filtered_ev, 
            x='kwhTotal',
            nbins=30,
            title='충전량 분포',
            labels={'kwhTotal': '충전량 (kWh)', 'count': '빈도'}
        )
        st.plotly_chart(fig1, use_container_width=True)
    
    with col2:
        # 충전 시간 vs 충전량
        fig2 = px.scatter(
            filtered_ev,
            x='chargeTimeHrs',
            y='kwhTotal',
            color='platform',
            title='충전 시간 vs 충전량',
            labels={'chargeTimeHrs': '충전 시간 (시간)', 'kwhTotal': '충전량 (kWh)'}
        )
        st.plotly_chart(fig2, use_container_width=True)

with tab2:
    # 요일별 패턴
    weekday_summary = filtered_ev[weekday_cols].sum().reset_index()
    weekday_summary.columns = ['Day', 'Sessions']
    
    fig3 = px.bar(
        weekday_summary,
        x='Day',
        y='Sessions',
        title='요일별 충전 세션 수',
        color='Sessions',
        color_continuous_scale='viridis'
    )
    st.plotly_chart(fig3, use_container_width=True)
    
    # 시간대별 패턴
    hourly_data = filtered_ev['startTime'].value_counts().sort_index().reset_index()
    hourly_data.columns = ['Hour', 'Sessions']
    
    fig4 = px.line(
        hourly_data,
        x='Hour',
        y='Sessions',
        title='시간대별 충전 시작 패턴',
        markers=True
    )
    st.plotly_chart(fig4, use_container_width=True)

with tab3:
    # 위치별 통계
    location_stats = filtered_ev.groupby('locationId').agg({
        'kwhTotal': ['sum', 'mean', 'count']
    }).round(2)
    
    st.subheader('위치별 충전 통계 (상위 10개)')
    st.dataframe(location_stats.head(10))
'''

print("\n스트림릿용 EV 충전 시각화 코드:")
print(streamlit_ev_code)

=== EV 충전 데이터 기본 분석 ===
총 충전 세션: 3,395건
총 충전량: 19,723.7 kWh
평균 충전량: 5.81 kWh
평균 충전 시간: 2.84 시간

스트림릿용 EV 충전 시각화 코드:

# EV 충전 데이터 시각화 (Streamlit)
st.header('⚡ 전기차 충전 패턴 분석')

# 필터링 옵션
st.sidebar.subheader('🔧 필터 설정')
min_kwh = st.sidebar.slider('최소 충전량 (kWh)', 0.0, float(ev_charge['kwhTotal'].max()), 0.0)
selected_platform = st.sidebar.multiselect(
    '플랫폼 선택',
    ev_charge['platform'].unique(),
    default=ev_charge['platform'].unique()
)

# 데이터 필터링
filtered_ev = ev_charge[
    (ev_charge['kwhTotal'] >= min_kwh) &
    (ev_charge['platform'].isin(selected_platform))
]

# 탭으로 구분된 시각화
tab1, tab2, tab3 = st.tabs(["📊 기본 분석", "📈 패턴 분석", "🗺️ 위치 분석"])

with tab1:
    col1, col2 = st.columns(2)

    with col1:
        # 충전량 분포
        fig1 = px.histogram(
            filtered_ev, 
            x='kwhTotal',
            nbins=30,
            title='충전량 분포',
            labels={'kwhTotal': '충전량 (kWh)', 'count': '빈도'}
        )
        st.plotly_chart(fig1, use_container_width=True)

    with col2

### 5.3 의료비 데이터 시각화

In [ ]:
# 의료비 데이터 분석
print("=== 의료비 데이터 기본 분석 ===")
print(f"총 데이터: {len(medical_cost):,}건")
print(f"평균 의료비: ${medical_cost['charges'].mean():,.2f}")
print(f"중앙값 의료비: ${medical_cost['charges'].median():,.2f}")
print(f"최고 의료비: ${medical_cost['charges'].max():,.2f}")
print(f"\n성별 분포: {medical_cost['sex'].value_counts().to_dict()}")
print(f"흡연자 비율: {medical_cost['smoker'].value_counts()['yes'] / len(medical_cost) * 100:.1f}%")

# 통계적 시각화
fig, axes = plt.subplots(2, 3, figsize=(18, 12))
fig.suptitle('Medical Cost Analysis', fontsize=16, fontweight='bold')

# 1. 의료비 분포
sns.histplot(medical_cost['charges'], bins=50, kde=True, ax=axes[0,0])
axes[0,0].set_title('Distribution of Medical Charges')
axes[0,0].set_xlabel('Charges ($)')

# 2. 나이 vs 의료비
sns.scatterplot(data=medical_cost, x='age', y='charges', 
                hue='smoker', alpha=0.7, ax=axes[0,1])
axes[0,1].set_title('Age vs Medical Charges (by Smoking Status)')

# 3. BMI vs 의료비
sns.scatterplot(data=medical_cost, x='bmi', y='charges', 
                hue='sex', alpha=0.7, ax=axes[0,2])
axes[0,2].set_title('BMI vs Medical Charges (by Gender)')

# 4. 흡연 여부별 의료비 박스플롯
sns.boxplot(data=medical_cost, x='smoker', y='charges', ax=axes[1,0])
axes[1,0].set_title('Medical Charges by Smoking Status')

# 5. 지역별 의료비
sns.boxplot(data=medical_cost, x='region', y='charges', ax=axes[1,1])
axes[1,1].set_title('Medical Charges by Region')
axes[1,1].tick_params(axis='x', rotation=45)

# 6. 자녀 수 vs 의료비
sns.boxplot(data=medical_cost, x='children', y='charges', ax=axes[1,2])
axes[1,2].set_title('Medical Charges by Number of Children')

plt.tight_layout()
plt.show()

# 상관관계 히트맵
plt.figure(figsize=(10, 8))
# 범주형 변수를 숫자로 변환
medical_encoded = medical_cost.copy()
medical_encoded['sex'] = medical_encoded['sex'].map({'male': 1, 'female': 0})
medical_encoded['smoker'] = medical_encoded['smoker'].map({'yes': 1, 'no': 0})
medical_encoded = pd.get_dummies(medical_encoded, columns=['region'])

correlation_matrix = medical_encoded.corr()
sns.heatmap(correlation_matrix, annot=True, cmap='coolwarm', center=0,
            square=True, linewidths=0.5)
plt.title('Correlation Matrix - Medical Cost Factors')
plt.tight_layout()
plt.show()

# 스트림릿용 의료비 시각화 코드
streamlit_medical_code = '''
# 의료비 데이터 시각화 (Streamlit)
st.header('🏥 의료비 영향 요인 분석')

# 사이드바 필터
st.sidebar.subheader('👥 인구통계학적 필터')
age_filter = st.sidebar.slider('나이 범위', 18, 64, (18, 64))
gender_filter = st.sidebar.multiselect('성별', ['male', 'female'], default=['male', 'female'])
smoker_filter = st.sidebar.multiselect('흡연 여부', ['yes', 'no'], default=['yes', 'no'])
region_filter = st.sidebar.multiselect(
    '지역', 
    medical_cost['region'].unique(), 
    default=medical_cost['region'].unique()
)

# 데이터 필터링
filtered_medical = medical_cost[
    (medical_cost['age'].between(age_filter[0], age_filter[1])) &
    (medical_cost['sex'].isin(gender_filter)) &
    (medical_cost['smoker'].isin(smoker_filter)) &
    (medical_cost['region'].isin(region_filter))
]

# 메인 대시보드
col1, col2, col3, col4 = st.columns(4)

with col1:
    avg_cost = filtered_medical['charges'].mean()
    st.metric('평균 의료비', f'${avg_cost:,.0f}')

with col2:
    median_cost = filtered_medical['charges'].median()
    st.metric('중앙값', f'${median_cost:,.0f}')

with col3:
    total_patients = len(filtered_medical)
    st.metric('환자 수', f'{total_patients:,}명')

with col4:
    smoker_ratio = filtered_medical['smoker'].value_counts().get('yes', 0) / len(filtered_medical) * 100
    st.metric('흡연자 비율', f'{smoker_ratio:.1f}%')

# 시각화 탭
tab1, tab2, tab3, tab4 = st.tabs(["📊 분포 분석", "🔍 요인 분석", "📈 상관관계", "🎯 예측 모델"])

with tab1:
    col1, col2 = st.columns(2)
    
    with col1:
        # 의료비 분포
        fig1 = px.histogram(
            filtered_medical,
            x='charges',
            nbins=30,
            title='의료비 분포',
            color_discrete_sequence=['skyblue']
        )
        st.plotly_chart(fig1, use_container_width=True)
    
    with col2:
        # 나이별 분포
        fig2 = px.histogram(
            filtered_medical,
            x='age',
            title='나이 분포',
            color_discrete_sequence=['lightgreen']
        )
        st.plotly_chart(fig2, use_container_width=True)

with tab2:
    # 흡연 여부별 의료비
    fig3 = px.box(
        filtered_medical,
        x='smoker',
        y='charges',
        color='smoker',
        title='흡연 여부별 의료비 분포'
    )
    st.plotly_chart(fig3, use_container_width=True)
    
    # 나이 vs 의료비 (흡연 여부로 색상 구분)
    fig4 = px.scatter(
        filtered_medical,
        x='age',
        y='charges',
        color='smoker',
        size='bmi',
        title='나이 vs 의료비 (흡연 여부별)',
        hover_data=['sex', 'children', 'region']
    )
    st.plotly_chart(fig4, use_container_width=True)

with tab3:
    # BMI vs 의료비
    fig5 = px.scatter(
        filtered_medical,
        x='bmi',
        y='charges',
        color='sex',
        facet_col='smoker',
        title='BMI vs 의료비 (성별 및 흡연 여부별)',
        trendline='ols'
    )
    st.plotly_chart(fig5, use_container_width=True)

with tab4:
    st.subheader('🤖 간단한 의료비 예측')
    
    # 사용자 입력
    col1, col2, col3 = st.columns(3)
    
    with col1:
        input_age = st.number_input('나이', 18, 100, 30)
        input_sex = st.selectbox('성별', ['male', 'female'])
    
    with col2:
        input_bmi = st.number_input('BMI', 15.0, 50.0, 25.0)
        input_children = st.number_input('자녀 수', 0, 10, 0)
    
    with col3:
        input_smoker = st.selectbox('흡연 여부', ['no', 'yes'])
        input_region = st.selectbox('지역', medical_cost['region'].unique())
    
    # 간단한 규칙 기반 예측
    if st.button('의료비 예측하기'):
        base_cost = 3000
        
        # 나이 요인
        age_factor = input_age * 50
        
        # BMI 요인
        if input_bmi > 30:
            bmi_factor = (input_bmi - 30) * 200
        else:
            bmi_factor = 0
        
        # 흡연 요인
        smoking_factor = 20000 if input_smoker == 'yes' else 0
        
        # 자녀 요인
        children_factor = input_children * 500
        
        predicted_cost = base_cost + age_factor + bmi_factor + smoking_factor + children_factor
        
        st.success(f'예상 의료비: ${predicted_cost:,.0f}')
        
        # 비교 데이터
        similar_data = filtered_medical[
            (abs(filtered_medical['age'] - input_age) <= 5) &
            (filtered_medical['sex'] == input_sex) &
            (filtered_medical['smoker'] == input_smoker)
        ]
        
        if len(similar_data) > 0:
            avg_similar = similar_data['charges'].mean()
            st.info(f'유사한 조건의 평균 의료비: ${avg_similar:,.0f}')
'''

print("\n스트림릿용 의료비 시각화 코드:")
print(streamlit_medical_code)

## 6. 통합 스트림릿 애플리케이션 코드

모든 데이터셋을 활용한 종합적인 대시보드

In [ ]:
# 통합 스트림릿 애플리케이션 생성
complete_streamlit_app = '''
# C:/githome/15week_cloud-data-viz/streamlit_dashboard.py
# 스트림릿 종합 데이터 시각화 대시보드
# 작성일: 2025년 8월 19일

import streamlit as st
import pandas as pd
import numpy as np
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import seaborn as sns
import matplotlib.pyplot as plt
from datetime import datetime, time
import warnings
warnings.filterwarnings('ignore')

# 페이지 설정
st.set_page_config(
    page_title="데이터 시각화 대시보드",
    page_icon="📊",
    layout="wide",
    initial_sidebar_state="expanded"
)

# CSS 스타일
st.markdown("""
<style>
.main {
    padding-top: 1rem;
}
.stMetric {
    background-color: #f0f2f6;
    border: 1px solid #e6e9ef;
    padding: 0.5rem;
    border-radius: 0.25rem;
}
</style>
""", unsafe_allow_html=True)

# 데이터 로드 함수
@st.cache_data
def load_data():
    """데이터 로드 및 전처리 함수"""
    data_path = "dataset/"
    
    # ABNB 주식 데이터
    abnb_stock = pd.read_csv(data_path + "ABNB_stock.csv")
    abnb_stock['Date'] = pd.to_datetime(abnb_stock['Date'])
    
    # EV 충전 데이터
    ev_charge = pd.read_csv(data_path + "EV_charge.csv")
    
    # 의료비 데이터
    medical_cost = pd.read_csv(data_path + "medical_cost.csv")
    
    return abnb_stock, ev_charge, medical_cost

# 메인 함수
def main():
    # 제목
    st.title("📊 종합 데이터 시각화 대시보드")
    st.markdown("##### 15주차 클라우드 기반 데이터 시각화 - 스트림릿 실습")
    
    # 데이터 로드
    try:
        abnb_stock, ev_charge, medical_cost = load_data()
        st.success("데이터 로드 완료! 📈")
    except Exception as e:
        st.error(f"데이터 로드 실패: {e}")
        return
    
    # 사이드바 - 전체 설정
    st.sidebar.title("🎛️ 대시보드 설정")
    
    # 페이지 선택
    page = st.sidebar.selectbox(
        "분석할 데이터 선택",
        ["📊 전체 개요", "📈 ABNB 주식", "⚡ EV 충전", "🏥 의료비", "🔧 스트림릿 구성요소"]
    )
    
    # 페이지별 렌더링
    if page == "📊 전체 개요":
        render_overview(abnb_stock, ev_charge, medical_cost)
    elif page == "📈 ABNB 주식":
        render_abnb_analysis(abnb_stock)
    elif page == "⚡ EV 충전":
        render_ev_analysis(ev_charge)
    elif page == "🏥 의료비":
        render_medical_analysis(medical_cost)
    elif page == "🔧 스트림릿 구성요소":
        render_streamlit_components()

def render_overview(abnb_stock, ev_charge, medical_cost):
    """전체 개요 페이지"""
    st.header("📊 데이터셋 전체 개요")
    
    # 메트릭 카드
    col1, col2, col3 = st.columns(3)
    
    with col1:
        st.metric(
            "📈 ABNB 주식 데이터",
            f"{len(abnb_stock):,}일",
            f"최신가: ${abnb_stock['Close'].iloc[-1]:.2f}"
        )
    
    with col2:
        st.metric(
            "⚡ EV 충전 세션",
            f"{len(ev_charge):,}건",
            f"총 {ev_charge['kwhTotal'].sum():,.0f} kWh"
        )
    
    with col3:
        st.metric(
            "🏥 의료비 데이터",
            f"{len(medical_cost):,}명",
            f"평균 ${medical_cost['charges'].mean():,.0f}"
        )
    
    st.divider()
    
    # 데이터셋 미리보기
    tab1, tab2, tab3 = st.tabs(["📈 ABNB Stock", "⚡ EV Charge", "🏥 Medical Cost"])
    
    with tab1:
        st.subheader("ABNB 주식 데이터 미리보기")
        st.dataframe(abnb_stock.head(), use_container_width=True)
        
        # 간단한 차트
        fig = px.line(abnb_stock, x='Date', y='Close', title='ABNB 주가 추이')
        st.plotly_chart(fig, use_container_width=True)
    
    with tab2:
        st.subheader("EV 충전 데이터 미리보기")
        st.dataframe(ev_charge.head(), use_container_width=True)
        
        # 플랫폼별 분포
        platform_counts = ev_charge['platform'].value_counts()
        fig = px.pie(values=platform_counts.values, names=platform_counts.index, 
                     title='플랫폼별 충전 세션 분포')
        st.plotly_chart(fig, use_container_width=True)
    
    with tab3:
        st.subheader("의료비 데이터 미리보기")
        st.dataframe(medical_cost.head(), use_container_width=True)
        
        # 흡연 여부별 의료비
        fig = px.box(medical_cost, x='smoker', y='charges', 
                     title='흡연 여부별 의료비 분포')
        st.plotly_chart(fig, use_container_width=True)

def render_abnb_analysis(abnb_stock):
    """ABNB 주식 분석 페이지"""
    st.header("📈 ABNB 주식 데이터 분석")
    
    # 필터링 옵션
    st.sidebar.subheader("📅 기간 설정")
    start_date = st.sidebar.date_input("시작일", abnb_stock['Date'].min().date())
    end_date = st.sidebar.date_input("종료일", abnb_stock['Date'].max().date())
    
    # 데이터 필터링
    filtered_data = abnb_stock[
        (abnb_stock['Date'].dt.date >= start_date) & 
        (abnb_stock['Date'].dt.date <= end_date)
    ]
    
    # 캔들스틱 차트
    fig = go.Figure(data=go.Candlestick(
        x=filtered_data['Date'],
        open=filtered_data['Open'],
        high=filtered_data['High'],
        low=filtered_data['Low'],
        close=filtered_data['Close']
    ))
    
    fig.update_layout(
        title='ABNB 주가 캔들스틱 차트',
        xaxis_title='날짜',
        yaxis_title='주가 ($)',
        height=500
    )
    
    st.plotly_chart(fig, use_container_width=True)
    
    # 거래량과 주가 상관관계
    col1, col2 = st.columns(2)
    
    with col1:
        fig_volume = px.bar(filtered_data, x='Date', y='Volume', 
                           title='일별 거래량')
        st.plotly_chart(fig_volume, use_container_width=True)
    
    with col2:
        # 일일 수익률 계산
        filtered_data = filtered_data.copy()
        filtered_data['Daily_Return'] = filtered_data['Close'].pct_change() * 100
        
        fig_return = px.line(filtered_data, x='Date', y='Daily_Return',
                            title='일일 수익률 (%)')
        fig_return.add_hline(y=0, line_dash="dash", line_color="red")
        st.plotly_chart(fig_return, use_container_width=True)

def render_ev_analysis(ev_charge):
    """EV 충전 분석 페이지"""
    st.header("⚡ 전기차 충전 패턴 분석")
    
    # 필터링 옵션
    st.sidebar.subheader("🔧 필터 설정")
    min_kwh = st.sidebar.slider("최소 충전량 (kWh)", 0.0, float(ev_charge['kwhTotal'].max()), 0.0)
    selected_platform = st.sidebar.multiselect(
        "플랫폼 선택",
        ev_charge['platform'].unique(),
        default=ev_charge['platform'].unique()
    )
    
    # 데이터 필터링
    filtered_data = ev_charge[
        (ev_charge['kwhTotal'] >= min_kwh) &
        (ev_charge['platform'].isin(selected_platform))
    ]
    
    # 기본 통계
    col1, col2, col3, col4 = st.columns(4)
    
    with col1:
        st.metric("총 세션 수", f"{len(filtered_data):,}")
    with col2:
        st.metric("총 충전량", f"{filtered_data['kwhTotal'].sum():,.1f} kWh")
    with col3:
        st.metric("평균 충전량", f"{filtered_data['kwhTotal'].mean():.2f} kWh")
    with col4:
        st.metric("평균 충전시간", f"{filtered_data['chargeTimeHrs'].mean():.2f} 시간")
    
    # 시각화
    tab1, tab2, tab3 = st.tabs(["📊 기본 분석", "⏰ 시간 패턴", "📍 위치 분석"])
    
    with tab1:
        col1, col2 = st.columns(2)
        
        with col1:
            fig1 = px.histogram(filtered_data, x='kwhTotal', nbins=30,
                               title='충전량 분포')
            st.plotly_chart(fig1, use_container_width=True)
        
        with col2:
            fig2 = px.scatter(filtered_data, x='chargeTimeHrs', y='kwhTotal',
                             color='platform', title='충전시간 vs 충전량')
            st.plotly_chart(fig2, use_container_width=True)
    
    with tab2:
        # 요일별 패턴
        weekdays = ['Mon', 'Tues', 'Wed', 'Thurs', 'Fri', 'Sat', 'Sun']
        weekday_data = []
        for day in weekdays:
            count = filtered_data[day].sum()
            weekday_data.append({'Day': day, 'Sessions': count})
        
        weekday_df = pd.DataFrame(weekday_data)
        fig3 = px.bar(weekday_df, x='Day', y='Sessions',
                     title='요일별 충전 세션 수')
        st.plotly_chart(fig3, use_container_width=True)
        
        # 시간대별 패턴
        hourly_data = filtered_data['startTime'].value_counts().sort_index().reset_index()
        hourly_data.columns = ['Hour', 'Sessions']
        
        fig4 = px.line(hourly_data, x='Hour', y='Sessions',
                      title='시간대별 충전 시작 패턴', markers=True)
        st.plotly_chart(fig4, use_container_width=True)
    
    with tab3:
        # 위치별 통계
        location_stats = filtered_data.groupby('locationId').agg({
            'kwhTotal': ['sum', 'mean', 'count']
        }).round(2)
        
        location_stats.columns = ['총_충전량', '평균_충전량', '세션_수']
        location_stats = location_stats.sort_values('세션_수', ascending=False)
        
        st.subheader("위치별 충전 통계 (상위 20개)")
        st.dataframe(location_stats.head(20), use_container_width=True)

def render_medical_analysis(medical_cost):
    """의료비 분석 페이지"""
    st.header("🏥 의료비 영향 요인 분석")
    
    # 필터링 옵션
    st.sidebar.subheader("👥 인구통계학적 필터")
    age_range = st.sidebar.slider("나이 범위", 18, 64, (18, 64))
    gender_filter = st.sidebar.multiselect("성별", ['male', 'female'], default=['male', 'female'])
    smoker_filter = st.sidebar.multiselect("흡연 여부", ['yes', 'no'], default=['yes', 'no'])
    region_filter = st.sidebar.multiselect(
        "지역", medical_cost['region'].unique(), default=medical_cost['region'].unique()
    )
    
    # 데이터 필터링
    filtered_data = medical_cost[
        (medical_cost['age'].between(age_range[0], age_range[1])) &
        (medical_cost['sex'].isin(gender_filter)) &
        (medical_cost['smoker'].isin(smoker_filter)) &
        (medical_cost['region'].isin(region_filter))
    ]
    
    # 기본 통계
    col1, col2, col3, col4 = st.columns(4)
    
    with col1:
        avg_cost = filtered_data['charges'].mean()
        st.metric("평균 의료비", f"${avg_cost:,.0f}")
    with col2:
        median_cost = filtered_data['charges'].median()
        st.metric("중앙값", f"${median_cost:,.0f}")
    with col3:
        total_patients = len(filtered_data)
        st.metric("환자 수", f"{total_patients:,}명")
    with col4:
        smoker_ratio = filtered_data['smoker'].value_counts().get('yes', 0) / len(filtered_data) * 100
        st.metric("흡연자 비율", f"{smoker_ratio:.1f}%")
    
    # 시각화 탭
    tab1, tab2, tab3, tab4 = st.tabs(["📊 분포 분석", "🔍 요인 분석", "📈 상관관계", "🎯 예측"])
    
    with tab1:
        col1, col2 = st.columns(2)
        
        with col1:
            fig1 = px.histogram(filtered_data, x='charges', nbins=30,
                               title='의료비 분포')
            st.plotly_chart(fig1, use_container_width=True)
        
        with col2:
            fig2 = px.histogram(filtered_data, x='age', nbins=20,
                               title='나이 분포')
            st.plotly_chart(fig2, use_container_width=True)
    
    with tab2:
        # 흡연 여부별 의료비
        fig3 = px.box(filtered_data, x='smoker', y='charges', color='smoker',
                     title='흡연 여부별 의료비 분포')
        st.plotly_chart(fig3, use_container_width=True)
        
        # 나이 vs 의료비
        fig4 = px.scatter(filtered_data, x='age', y='charges', color='smoker',
                         size='bmi', title='나이 vs 의료비 (흡연 여부별)',
                         hover_data=['sex', 'children', 'region'])
        st.plotly_chart(fig4, use_container_width=True)
    
    with tab3:
        # BMI vs 의료비
        fig5 = px.scatter(filtered_data, x='bmi', y='charges', color='sex',
                         facet_col='smoker', title='BMI vs 의료비 (성별 및 흡연 여부별)',
                         trendline='ols')
        st.plotly_chart(fig5, use_container_width=True)
    
    with tab4:
        st.subheader("🤖 간단한 의료비 예측")
        
        col1, col2, col3 = st.columns(3)
        
        with col1:
            input_age = st.number_input("나이", 18, 100, 30)
            input_sex = st.selectbox("성별", ['male', 'female'])
        
        with col2:
            input_bmi = st.number_input("BMI", 15.0, 50.0, 25.0)
            input_children = st.number_input("자녀 수", 0, 10, 0)
        
        with col3:
            input_smoker = st.selectbox("흡연 여부", ['no', 'yes'])
            input_region = st.selectbox("지역", medical_cost['region'].unique())
        
        if st.button("의료비 예측하기", type="primary"):
            # 간단한 규칙 기반 예측
            base_cost = 3000
            age_factor = input_age * 50
            bmi_factor = max(0, (input_bmi - 30) * 200)
            smoking_factor = 20000 if input_smoker == 'yes' else 0
            children_factor = input_children * 500
            
            predicted_cost = base_cost + age_factor + bmi_factor + smoking_factor + children_factor
            
            st.success(f"예상 의료비: ${predicted_cost:,.0f}")
            
            # 유사 데이터와 비교
            similar_data = filtered_data[
                (abs(filtered_data['age'] - input_age) <= 5) &
                (filtered_data['sex'] == input_sex) &
                (filtered_data['smoker'] == input_smoker)
            ]
            
            if len(similar_data) > 0:
                avg_similar = similar_data['charges'].mean()
                st.info(f"유사한 조건의 평균 의료비: ${avg_similar:,.0f}")

def render_streamlit_components():
    """스트림릿 구성요소 실습 페이지"""
    st.header("🔧 스트림릿 구성요소 실습")
    
    # 텍스트 요소
    st.subheader("📝 텍스트 출력 요소")
    
    with st.expander("텍스트 요소 예제 보기"):
        st.title("제목 (st.title)")
        st.header("헤더 (st.header)")
        st.subheader("서브헤더 (st.subheader)")
        
        st.markdown("""
        **마크다운 텍스트 (st.markdown)**
        - :red[빨간색] 텍스트
        - :blue[파란색] 텍스트
        - :green[초록색] 텍스트
        """)
        
        st.text("일반 텍스트 (st.text)")
        
        st.code("""
        # 코드 블록 (st.code)
        import streamlit as st
        st.write("Hello, Streamlit!")
        """, language='python')
    
    st.divider()
    
    # 입력 위젯
    st.subheader("🎛️ 사용자 입력 위젯")
    
    col1, col2, col3 = st.columns(3)
    
    with col1:
        st.write("**버튼 및 선택**")
        if st.button("클릭하세요!"):
            st.balloons()
        
        checkbox_val = st.checkbox("체크박스")
        if checkbox_val:
            st.write("체크되었습니다!")
        
        radio_val = st.radio("라디오 버튼", ["옵션 1", "옵션 2", "옵션 3"])
        st.write(f"선택: {radio_val}")
    
    with col2:
        st.write("**선택 및 입력**")
        selectbox_val = st.selectbox("선택박스", ["선택 1", "선택 2", "선택 3"])
        
        multiselect_val = st.multiselect("다중 선택", ["A", "B", "C", "D"])
        st.write(f"선택된 항목: {multiselect_val}")
        
        text_val = st.text_input("텍스트 입력", placeholder="여기에 입력하세요")
        if text_val:
            st.write(f"입력값: {text_val}")
    
    with col3:
        st.write("**숫자 및 범위**")
        number_val = st.number_input("숫자 입력", 0, 100, 50)
        st.write(f"숫자: {number_val}")
        
        slider_val = st.slider("슬라이더", 0, 100, 25)
        st.write(f"슬라이더 값: {slider_val}")
        
        range_val = st.slider("범위 슬라이더", 0, 100, (20, 80))
        st.write(f"범위: {range_val}")
    
    st.divider()
    
    # 데이터 표시
    st.subheader("📊 데이터 표시 요소")
    
    # 샘플 데이터 생성
    sample_data = pd.DataFrame({
        '이름': ['김철수', '이영희', '박민수', '정지원'],
        '나이': [25, 30, 35, 28],
        '점수': [85, 92, 78, 88],
        '등급': ['B', 'A', 'C', 'B']
    })
    
    tab1, tab2, tab3 = st.tabs(["📋 표", "📈 메트릭", "💾 JSON"])
    
    with tab1:
        st.write("**정적 테이블 (st.table)**")
        st.table(sample_data)
        
        st.write("**인터랙티브 데이터프레임 (st.dataframe)**")
        st.dataframe(sample_data, use_container_width=True)
    
    with tab2:
        st.write("**메트릭 카드 (st.metric)**")
        
        metric_col1, metric_col2, metric_col3 = st.columns(3)
        
        with metric_col1:
            st.metric("평균 점수", f"{sample_data['점수'].mean():.1f}", "3.2")
        
        with metric_col2:
            st.metric("총 인원", f"{len(sample_data)}명", "1")
        
        with metric_col3:
            st.metric("최고 점수", f"{sample_data['점수'].max()}", "5")
    
    with tab3:
        st.write("**JSON 데이터 (st.json)**")
        
        json_data = {
            "project": "Streamlit Dashboard",
            "version": "1.0.0",
            "author": "Data Scientist",
            "features": [
                "Interactive Widgets",
                "Data Visualization",
                "Real-time Updates"
            ],
            "config": {
                "theme": "light",
                "layout": "wide"
            }
        }
        
        st.json(json_data)
    
    st.divider()
    
    # 레이아웃
    st.subheader("📐 레이아웃 구성")
    
    with st.expander("컬럼 레이아웃 예제"):
        layout_col1, layout_col2, layout_col3 = st.columns([2, 1, 1])
        
        with layout_col1:
            st.write("**넓은 컬럼 (2:1:1 비율)**")
            st.info("이 컬럼은 다른 컬럼의 2배 크기입니다.")
        
        with layout_col2:
            st.write("**컬럼 2**")
            st.success("성공!")
        
        with layout_col3:
            st.write("**컬럼 3**")
            st.warning("주의!")
    
    # 상태 관리 예제
    st.subheader("🔄 상태 관리 (Session State)")
    
    with st.expander("카운터 예제"):
        if 'counter' not in st.session_state:
            st.session_state.counter = 0
        
        col1, col2, col3 = st.columns(3)
        
        with col1:
            if st.button("+ 1"):
                st.session_state.counter += 1
        
        with col2:
            st.write(f"**카운트: {st.session_state.counter}**")
        
        with col3:
            if st.button("리셋"):
                st.session_state.counter = 0

# 애플리케이션 실행
if __name__ == "__main__":
    main()
'''

print("\n=== 완전한 스트림릿 애플리케이션 코드 ===")
print("파일명: streamlit_dashboard.py")
print("실행 방법: streamlit run streamlit_dashboard.py")
print("\n코드가 준비되었습니다!")

## 7. 실행 가이드 및 추가 학습 리소스

### 7.1 스트림릿 애플리케이션 실행 방법

In [ ]:
# 실행 가이드
execution_guide = '''
📚 스트림릿 애플리케이션 실행 가이드

1. 환경 준비:
   pip install streamlit pandas numpy plotly seaborn matplotlib

2. 파일 저장:
   - 위의 complete_streamlit_app 코드를 'streamlit_dashboard.py'로 저장
   - 데이터셋 파일들이 'dataset/' 폴더에 있는지 확인

3. 애플리케이션 실행:
   streamlit run streamlit_dashboard.py

4. 브라우저에서 확인:
   - 자동으로 브라우저가 열림 (http://localhost:8501)
   - 실시간으로 코드 수정 사항이 반영됨

5. 주요 기능:
   - 📊 전체 개요: 모든 데이터셋의 요약 정보
   - 📈 ABNB 주식: 주가 분석 및 캔들스틱 차트
   - ⚡ EV 충전: 충전 패턴 및 시간대별 분석
   - 🏥 의료비: 영향 요인 분석 및 간단한 예측
   - 🔧 스트림릿 구성요소: 모든 위젯 실습

6. 개발 팁:
   - Ctrl+S로 저장하면 자동 새로고침
   - st.cache_data로 데이터 로딩 최적화
   - 사이드바를 활용한 컨트롤 패널 구성
   - 탭과 컬럼으로 레이아웃 최적화
'''

print(execution_guide)

### 7.2 학습 정리 및 다음 단계

In [ ]:
# 학습 정리
learning_summary = '''
🎯 오늘 학습한 스트림릿 구성 요소

1. 텍스트 출력 요소 (Text Elements):
   ✅ st.title, st.header, st.subheader
   ✅ st.markdown, st.text
   ✅ st.code, st.divider

2. 데이터 표시 요소 (Data Display):
   ✅ st.dataframe, st.table
   ✅ st.metric (KPI 표시)
   ✅ st.json

3. 사용자 입력 위젯 (Input Widgets):
   ✅ st.button, st.checkbox, st.toggle
   ✅ st.radio, st.selectbox, st.multiselect
   ✅ st.slider, st.text_input, st.number_input

4. 레이아웃 구성 (Layout):
   ✅ st.columns - 컬럼 레이아웃
   ✅ st.tabs - 탭 구성
   ✅ st.sidebar - 사이드바
   ✅ st.expander - 확장 가능한 섹션

5. 상태 관리 (State Management):
   ✅ st.session_state - 세션 상태 관리
   ✅ key 매개변수 - 위젯 식별

6. 캐싱 (Caching):
   ✅ @st.cache_data - 데이터 캐싱
   ✅ 성능 최적화

7. 실제 데이터 활용:
   ✅ ABNB 주식 데이터 - 금융 데이터 시각화
   ✅ EV 충전 데이터 - IoT 패턴 분석
   ✅ 의료비 데이터 - 통계적 분석 및 예측

🚀 다음 단계 학습 방향:

1. 고급 스트림릿 기능:
   - 커스텀 컴포넌트 개발
   - 멀티페이지 애플리케이션
   - 파일 업로드/다운로드

2. 배포 및 운영:
   - Streamlit Community Cloud 배포
   - Docker 컨테이너화
   - 보안 및 인증

3. 고급 시각화:
   - Plotly Dash 연동
   - 3D 시각화
   - 실시간 데이터 스트리밍

4. 머신러닝 통합:
   - 모델 서빙
   - A/B 테스트 프레임워크
   - 모델 모니터링 대시보드
'''

print(learning_summary)
print("\n" + "="*60)
print("🎉 스트림릿 학습 실습이 완료되었습니다!")
print("📂 생성된 파일: 250819_streamlit.ipynb")
print("🌟 이제 실제 대시보드를 만들어보세요!")
print("="*60)